In [ ]:
!git clone https://github.com/Swastik166/TANS_trial.git
    

In [ ]:
import sys
sys.path.insert(0, './TANS_trial')
!pip install config

In [ ]:
%%bash
cd TANS_trial
python3 main.py --gpu 0 --mode train --batch-size 140 --n-epochs 10000 --base-path path/for/storing/outcomes/ --data-path path/to/processed/dataset/is/stored/ --model-zoo path/to/model_zoo.pt --seed 777 

In [ ]:
import torch
import numpy as np
import random
import torchvision.models as models
"""add utils from misc and use the finctions available in for better error handling"""

class ModelZoo:
    def __init__(self, datasets, seed):
        self.seed = seed
        self.datasets = ['dataset1']
        self.models = {}
        self.train_instances = {}
        self.noise = torch.load('/path/to/noise.pt')
        torch.cuda.manual_seed(self.seed)
        torch.manual_seed(self.seed)
        np.random.seed(self.seed)
        random.seed(self.seed)

    def init_loaders(self):
        # Get loaders for train, test, and validation data
        self.tr_dataset, self.tr_loader = get_loader(dataset, 'train')
        self.te_dataset, self.te_loader = get_loader(dataset, 'test')
        self.val_dataset, self.val_loader = get_loader(dataset, 'validation')
        self.nclass = self.tr_dataset.get_nclss()
        
    def create_zoo(self):
        model_zoo_dict = {}

        for dataset in self.datasets:
            # Initializing loaders for the dataset
            self.init_loaders(dataset)

            acc_list = []
            n_params_list = []
            topol_list = []

            for _ in range(10):
                # Get neural network model and topology information
                topol, net = self.get_net(ncalss)

                # Training the model and obtaining accuracy
                acc = self.train(net, dataset, data_loader)  # Assuming train method returns accuracy
                acc_list.append(acc)

                # Calculating number of parameters
                n_params = self.n_param(net)
                n_params_list.append(n_params)

                topol_list.append(topol)

            # Save values for this dataset in the model_zoo_dict
            model_zoo_dict[dataset] = {
                'acc': acc_list,
                'n_params': n_params_list,
                'topol': topol_list
            }

        torch.save(model_zoo_dict, 'model_zoo.pt')

    def get_net(self,nclss):
        super_net_name = "ofa_supernet_mbv3_w10"

        # Load the super network from the 'mit-han-lab/once-for-all' repository
        super_net = torch.hub.load('mit-han-lab/once-for-all', super_net_name, pretrained=True).eval()

        # Sample an active subnet configuration from the super network
        sampled_config = super_net.sample_active_subnet()

        # Extract topology information from the sampled configuration
        pre_topol = list(sampled_config.values())
        topol = []

        for i in pre_topol:
            for j in i:
                topol.append(j)

        # Split the topology into kernel sizes, expansion ratios, and depths
        ks = topol[:20] 
        e = topol[20:40]
        d = topol[40:]

        # Set the active subnet in the super network using the sampled topology
        super_net.set_active_subnet(ks=ks, e=e, d=d)
        active_subnet = super_net.get_active_subnet(preserve_weight=True)
        active_subnet.classifier = torch.nn.Linear(1536, nclss)
        active_subnet = active_subnet.to(self.device)

        return topol, active_subnet

    def f_emb(self, dataset, net):
        model = net        
        module = list(model.children())[:-1]
        model = torch.nn.Sequential(*module)
        model.eval()
        with torch.no_grad():
            # Forward pass through ResNet18
            f_emb = model(self.noise)
        print(f_emb)
        return f_emb


    def train(self, model, train_loader, nclss):
        optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=4e-5)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, float(self.args.n_eps_finetuning))
        pass  # Replace with implementation for training

    def n_param(self, model):
        # Calculate the number of parameters in the model
        return sum(p.numel() for p in model.parameters() if p.requires_grad)

    def get_query(self, dataset):
        model = models.resneth18(pretrained = True)
        module = list(model.children())[:-1]
        model = torch.nn.Sequential(*module)
        model.eval()
        with torch.no_grad():
            images, _ =  next(iter(self.tr_loader))
            print(len(images))
            # Forward pass through ResNet18
            query_train = model(images)
        print (query_train.shape)
        with torch.no_grad():
            images, _ =  next(iter(self.te_loader))
            print(len(images))
            # Forward pass through ResNet18
        query_test = model(images)
        print (query_test.shape)
        return query_train, query_test

    def save_zoo(self):
        # Save model zoo to model_zoo.pt
        model_zoo_dict = {
            'dataset': self.datasets,
            'acc': {},  # Placeholder for accuracy (fill this with values)
            'n_params': {dataset: self.n_param(self.models[dataset]) for dataset in self.datasets},
            'topol': {},  # Placeholder for topology information
            'f_emb': {}  # Placeholder for feature embeddings
        }
        torch.save(model_zoo_dict, 'model_zoo.pt')

    def save_trainpt(self):
        # Save training instances to m_train.pt
        trainpt_dict = {}
        for dataset in self.datasets:
            trainpt_dict[dataset] = {
                'task': {},  # Placeholder for task information
                'clss': {},  # Placeholder for class information
                'nclss': {},  # Placeholder for number of classes
                'x_query_train': {},  # Placeholder for training query data
                'x_query_test': {}  # Placeholder for testing query data
            }
        torch.save(trainpt_dict, 'm_train.pt')
